# The Idea in General

We have a preconcieved notion of what families or communities of people are like, in regards to their temper or sentiment they give off. We now want to investigate if it holds up to a more robust and "objective" analysis of these communities' sentiments. A network describing each person's relation to another based on costarring in a scene will also include a color attribute that tells us if the person is more negative or positive based on their dialogue throughout the series.  
As such we will see clusterings of communities and can gather their coloring to determine their overall sentiments. A non visualized attribute will be the birthing place of the individual characters/nodes, maybe there's just a place that breeds negativity?  
In addition, we will present the 5 biggest families and their sentiments as to also hold them against our previous beliefs.

- Other text analysis? Most used words by each family/community?

### Steps to take
1. win
2. win
3. win some more